In [1]:
import sys
from bs4 import BeautifulSoup

sys.path.insert(0, '../../src')

from eeyore_nlp.pumps import WebPump, ContextPump
from eeyore_nlp.pipelines import ContextTokenizer, TextPipeline, TextPipe

In [2]:
class EspnContentScrapper(TextPipe):
    def __init__(self):
        super().__init__(1)
    
    def execute(self, text: str) -> str:
        bs = BeautifulSoup(text, 'html.parser')

        elements_to_remove = [
            bs.find_all('ul', 'article-social'),
            bs.find_all('div', 'article-meta'),
            bs.find_all('aside'),
            bs.find_all('div', 'teads-inread'),
            bs.find_all('figure'),
            bs.find_all('div', 'cookie-overlay')
        ]

        for element_search in elements_to_remove:
            for tag in element_search:
                tag.decompose()

        for a in bs.find_all('a'):
            a.replaceWith(a.text)

        p = [ p.text for p in bs.find_all('p') ]

        return '\n'.join(p).strip()

In [3]:
context_pump = ContextPump(
    WebPump([
        'https://www.espn.com/nfl/story/_/id/31433376/',
    ]),
    ContextTokenizer(
        text_preprocessor=TextPipeline(pipes=[
            EspnContentScrapper()
        ])
    )
)

generator = context_pump.execute()

In [4]:
print(next(generator).sentence)
print()
print(next(generator).sentence)
print()
print(next(generator).sentence)

Former Carolina quarterback Teddy Bridgewater on Wednesday questioned how the Panthers practiced in two critical areas that played a part in the organization moving on from him after the first year of a three-year contract.

"I'll just say this, for Joe Brady's growth they'll have to practice different things in different ways," Bridgewater said of Carolina's offensive coordinator on the "All Things Covered" CBS Sports podcast with Patrick Peterson and Bryant McFadden.

"One of the things we didn't do much of when I was there, we didn't practice two-minute drills, we didn't practice red zone.
